# IMPORT THƯ VIỆN


In [224]:
!pip install pyvi
!pip install underthesea

In [225]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [226]:
from pandas import read_excel
from underthesea import word_tokenize

my_sheet = 'Sheet1'
file_name_train = '/content/drive/MyDrive/DATAMINING/train_nor_811.xlsx'
file_name_val = '/content/drive/MyDrive/DATAMINING/valid_nor_811.xlsx'

In [227]:
import re
import string
import pandas as pd
from pyvi import ViTokenizer
from underthesea import pos_tag
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
max_fatures = 2000

# ĐỌC FILE TRAIN VS VALIDATE

In [228]:
df = read_excel(file_name_train, sheet_name = my_sheet)
df = df.drop(['Unnamed: 0'], axis=1)
print(df.head())
X_train = df['Sentence']
y_train = df['Emotion']

     Emotion                                           Sentence
0      Other              cho mình xin bài nhạc tên là gì với ạ
1    Disgust  cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2    Disgust  lo học đi . yêu đương lol gì hay lại thích học...
3  Enjoyment    uớc gì sau này về già vẫn có thể như cụ này :))
4  Enjoyment  mỗi lần có video của con là cứ coi đi coi lại ...


In [229]:
df = read_excel(file_name_val, sheet_name = my_sheet)
df =df.drop(['Unnamed: 0'], axis=1)
print(df.head())
X_val = df['Sentence']
y_val = df['Emotion']

     Emotion                                           Sentence
0      Other  tính tao tao biết , chẳng có chuyện gì có thể ...
1  Enjoyment           lại là lào cai , tự hào quê mình quá :))
2    Sadness                                     bị từ chối rồi
3  Enjoyment                         tam đảo trời đẹp các mem à
4      Other  đọc bình luận của thằng đó không thiếu chữ nào 😂😂


# PREPROCESSING DATA

In [230]:
def normalize_text(text):
  #Remove các ký tự kéo dài: vd: đẹppppppp
  text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)
  text = text.replace("\n", " ")
  text = text.lower()
 
  # chuyen punctuation thành space
  translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
  text = text.translate(translator)

  text = ViTokenizer.tokenize(text)
  texts = text.split()
  texts = [t.replace('_', ' ') for t in texts]
  text = u' '.join(texts)

  #remove nốt những ký tự thừa thãi
  text = text.replace(u'"', u' ')
  text = text.replace(u'️', u'')
  text = text.replace('🏻','')

  #segmentation
  text = word_tokenize(text, format="text")
  return text

In [231]:
X_train = X_train.map(lambda x: normalize_text(x))
X_train[:10]

0                cho mình xin bài nhạc tên là gì với ạ
1    cho đáng đời con quỷ về nhà lôi con nhà mày ra...
2    lo học đi yêu_đương lol gì hay lại thích học_s...
3          uớc gì sau_này về già vẫn có_thể như cụ này
4    mỗi lần có video của con là cứ coi đi coi lại ...
5    thằng kia sao mày bắt vợ với bồ tao dọn thế ki...
6                        một lí_do trog muôn_vàn lí_do
7               thật hay đùa ác vậy không_thể tin được
8    ko phải con mình mà xem còn thấy đau như vậy h...
9    per nghe đi rồi khóc 1 trận cho thoải_mái đừng...
Name: Sentence, dtype: object

In [232]:
X_val = X_val.map(lambda x: normalize_text(x))
X_val[:10]

0    tính tao tao biết chẳng có chuyện gì có_thể là...
1                   lại là lào cai tự_hào quê mình quá
2                                       bị từ_chối rồi
3                           tam_đảo trời đẹp các mem à
4    đọc bình_luận của thằng đó không thiếu chữ nào...
5                        crush tao vẫn còn trinh nhé 😘
6    nó nói không đúg ư đg của nhà bà ư nó đóg phí ...
7                    gap kiểu này chắc đái ra quần quá
8                                   yêu em mọa mọa 😘 😘
9    có ai như tao không đọc đề thì quen nhưng lại ...
Name: Sentence, dtype: object

# LẤY VALIDATE & TRAIN DATASET LÀM TẬP TRAIN

In [233]:
X = X_train.append(X_val,ignore_index=True)
y = y_train.append(y_val,ignore_index=True)

# VECTOR HOÁ FEATURE & LABEL

## feature - X

In [234]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(
    binary=True, ngram_range=(1, 2),
    sublinear_tf=True,
    use_idf=True,
)

vectorizer.fit(X)
X = vectorizer.transform(X)
print(X)

  (0, 42026)	0.3194230222694976
  (0, 42025)	0.22273969491569742
  (0, 41375)	0.155019455465049
  (0, 38054)	0.30878650685603176
  (0, 38046)	0.23684005876057176
  (0, 25821)	0.3344143487545289
  (0, 25803)	0.2425394965740062
  (0, 21388)	0.2937951803710005
  (0, 21209)	0.14012453884016166
  (0, 17504)	0.2492805007405798
  (0, 17349)	0.1047146669770741
  (0, 12588)	0.3194230222694976
  (0, 12425)	0.12917560617270482
  (0, 3833)	0.2492805007405798
  (0, 3707)	0.1258152135705541
  (0, 1383)	0.2937951803710005
  (0, 1365)	0.20041101003859368
  (1, 47293)	0.24631804353097653
  (1, 47282)	0.1519997076196926
  (1, 44528)	0.16726801349233955
  (1, 44525)	0.25787836934434744
  (1, 44497)	0.13842598516918578
  (1, 41244)	0.20239799661490518
  (1, 41178)	0.12834325794033932
  (1, 30644)	0.21835335432502892
  :	:
  (6233, 20984)	0.17325919566905473
  (6233, 20829)	0.07696035242838022
  (6233, 18731)	0.17325919566905473
  (6233, 18702)	0.10747877627415438
  (6233, 18025)	0.14444754435215154
  (623

## Label - Y

In [235]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y.values)
y = le.transform(y.values)

In [236]:
print(y)

[4 1 1 ... 1 6 0]


# MODEL

In [237]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

Logistic = LogisticRegression(max_iter=1000)
Logistic.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# TEST DATASET

In [238]:
my_sheet = 'Sheet1'
file_name_test = '/content/drive/MyDrive/DATAMINING/test_nor_811.xlsx'

In [239]:
df = read_excel(file_name_test, sheet_name = my_sheet)
df =df.drop(['Unnamed: 0'], axis=1)
print(df.head())
X_test = df['Sentence']
y_test = df['Emotion']

    Emotion                                           Sentence
0   Sadness                   người ta có bạn bè nhìn vui thật
1  Surprise          cho nghỉ viêc mói đúng sao goi là kỷ luật
2   Disgust                                         kinh vãi 😡
3      Fear  nhà thì không xa lắm nhưng chưa bao giờ đi vì ...
4     Anger      bố không thích nộp đấy mày thích ý kiến không


In [240]:
X_test = vectorizer.transform(X_test)
y_test = le.transform(y_test.values)

In [241]:
y_logistic_predict = Logistic.predict(X_test)
print(classification_report(y_test, y_logistic_predict))
print(accuracy_score(y_test, y_logistic_predict))

              precision    recall  f1-score   support

           0       0.50      0.10      0.17        40
           1       0.43      0.54      0.48       132
           2       0.48      0.71      0.57       193
           3       0.90      0.39      0.55        46
           4       0.42      0.37      0.40       129
           5       0.60      0.50      0.55       116
           6       0.67      0.11      0.19        37

    accuracy                           0.49       693
   macro avg       0.57      0.39      0.41       693
weighted avg       0.52      0.49      0.47       693

0.4906204906204906


# CRAWL DATASET

In [242]:
df = pd.read_csv('/content/drive/MyDrive/DATAMINING/GFG_1.csv', encoding= 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis=1)
print(df.head())
X_test_crawl_1 = df['comment']
y_test_crawl_1 = df['label']

df = pd.read_csv('/content/drive/MyDrive/DATAMINING/GFG_2.csv', encoding= 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis=1)
print(df.head())
X_test_crawl_2 = df['comment']
y_test_crawl_2 = df['label']

df = pd.read_csv('/content/drive/MyDrive/DATAMINING/GFG_3.csv', encoding= 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis=1)
print(df.head())
X_test_crawl_3 = df['comment']
y_test_crawl_3 = df['label']

                                             comment      label
0  Dù cho tất cả thế giới có quay lưng với em anh...  Enjoyment
1  thằng bạn t bị câm từ nhỏ nghe xong bài này nó...    Disgust
2                   Có ai từ Mv Răng khôn qua đây ko  Enjoyment
3  Cơm sườn hay bún mắm\nAnh mún ăn cơm sườn hay ...  Enjoyment
4              Tự nhiên t thấy yêu Chi Pu ghê =)))))  Enjoyment
                                             comment      label
0  05:45  Có những cuộc gặp gỡ chớp nhoáng...      Other
1  Từ đoạn Hiền Hồ cao trào và luôn cả đoạn cuối ...  Enjoyment
2  Một sự kết hợp quá tuyệt vờiiiiiiii.. Hy vọng ...  Enjoyment
3  Hai ông bà này có điểm chung là\n- Hát hay\n- ...  Enjoyment
4  03:41  Anh ghét em lắm em biết không? Vì suốt ...    Sadness
                                             comment    label
0        Ngồi sau xe bạn hiền cảm giác ấm áp lạ kỳ..    Other
1  Mỗi lần mệt mỏi nghe nhạc cảm thấy bình yên đế...    Other
2  Mỗi lần nghe bài này đều khóc, khóc là vì k

In [243]:
X_test_crawl_1 = vectorizer.transform(X_test_crawl_1)
y_test_crawl_1 = le.transform(y_test_crawl_1.values)

X_test_crawl_2 = vectorizer.transform(X_test_crawl_2)
y_test_crawl_2 = le.transform(y_test_crawl_2.values)

X_test_crawl_3 = vectorizer.transform(X_test_crawl_3)
y_test_crawl_3 = le.transform(y_test_crawl_3.values)

In [244]:
y_crawl_logistic_predict_1 = Logistic.predict(X_test_crawl_1)
print(classification_report(y_test_crawl_1, y_crawl_logistic_predict_1))
print(accuracy_score(y_test_crawl_1, y_crawl_logistic_predict_1))

y_crawl_logistic_predict_2 = Logistic.predict(X_test_crawl_2)
print(classification_report(y_test_crawl_2, y_crawl_logistic_predict_2))
print(accuracy_score(y_test_crawl_2, y_crawl_logistic_predict_2))

y_crawl_logistic_predict_3 = Logistic.predict(X_test_crawl_3)
print(classification_report(y_test_crawl_3, y_crawl_logistic_predict_3))
print(accuracy_score(y_test_crawl_3, y_crawl_logistic_predict_3))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.29      0.80      0.42         5
           2       0.49      0.88      0.63        41
           3       0.00      0.00      0.00         7
           4       0.41      0.37      0.39        30
           5       0.83      0.42      0.56        12
           6       0.00      0.00      0.00        17

    accuracy                           0.47       120
   macro avg       0.29      0.35      0.28       120
weighted avg       0.37      0.47      0.39       120

0.4666666666666667
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.47      0.96      0.63        45
           4       0.33      0.10      0.15        21
           5       0.33      0.60      0.43         5
           6       0.00      0.00      0.00        38

    accuracy                           0.44       109
   m

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
